In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# !pip install gtts pydub audioop-lts
from pydub import AudioSegment
import time
from datetime import date
import os
import copy
import pandas as pd
from gtts import gTTS
from collections import defaultdict
from utils_data import (
    load_raw_data, check_dups, filter_df_to_vocab_of_interest, 
    fill_default_settings
    )
from utils_audio import (
    create_tts_files_for_one_vocab_word, compute_pinyin_and_create_recordings,
    combine_audio_files_and_compute_durations
)
from video_file_configs.a002_second_refresh import dict_recordings
pd.options.display.max_columns = 100
pd.options.display.max_rows = 130


# 0. Create dataframe of settings

In [3]:
audio_final_path = 'audio_files/products/audio_refresh/' + date.today().strftime('%Y%m%d') + '/'
if not os.path.exists(audio_final_path):
    os.mkdir(audio_final_path)
print(audio_final_path)

audio_files/products/audio_refresh/20250922/


In [4]:
# Create settings df
df_all_recordings_tomake = fill_default_settings(dict_recordings)
df_all_recordings_tomake

,recording_id,filename_suffix,min_priority,max_priority,min_known_english_prompt,max_known_english_prompt,min_known_pinyin_prompt,max_known_pinyin_prompt,types_allowed,min_combo_quality,min_date,sort_keys,sort_asc,category_type,cat1_values_allowed,types_allowed_str,min_adu,min_per,contains_character,exclude_words,max_count,categories_allowed,recording_name
0,012,efirst_p1_k345,1,1,3.0,5.0,1.0,6.0,"[combo, no combo, two word, prefix, single cha...",6.0,2025-01-01,"[category1, category2, pinyin]","[True, True, True]",,None,,1,1,None,None,1000000,None,0922_012_efirst_p1_k345
1,012,efirst_p2_k345,2,2,3.0,5.0,1.0,6.0,"[combo, no combo, two word, prefix, single cha...",6.0,2025-01-01,"[category1, category2, pinyin]","[True, True, True]",,None,,1,1,None,None,1000000,None,0922_012_efirst_p2_k345
2,012,efirst_p3_k345,3,3,3.0,5.0,1.0,6.0,"[combo, no combo, two word, prefix, single cha...",6.0,2025-01-01,"[category1, category2, pinyin]","[True, True, True]",,None,,1,1,None,None,1000000,None,0922_012_efirst_p3_k345
3,012,_p123_k2,1,3,2.0,2.0,1.0,6.0,"[combo, no combo, two word, prefix, single cha...",6.0,2025-01-01,"[category1, category2, pinyin]","[True, True, True]",,None,,1,1,None,None,1000000,None,0922_012__p123_k2
4,001,_p1_k345,1,1,1.0,6.0,3.0,5.0,"[combo, no combo, two word, prefix, single cha...",6.0,2025-01-01,"[category1, category2, pinyin]","[True, True, True]",,None,,1,1,None,None,1000000,None,0922_001__p1_k345
5,001,_p2_k345,2,2,1.0,6.0,3.0,5.0,"[combo, no combo, two word, prefix, single cha...",6.0,2025-01-01,"[category1, category2, pinyin]","[True, True, True]",,None,,1,1,None,None,1000000,None,0922_001__p2_k345
6,001,_p3_k345,3,3,1.0,6.0,3.0,5.0,"[combo, no combo, two word, prefix, single cha...",6.0,2025-01-01,"[category1, category2, pinyin]","[True, True, True]",,None,,1,1,None,None,1000000,None,0922_001__p3_k345
7,016,_phrase_p123,1,3,2.0,5.0,1.0,6.0,"[phrase, part sent, phrase_save, speak_phrase,...",6.0,2025-01-01,"[category1, category2, pinyin]","[True, True, True]",,None,,1,1,None,None,1000000,None,0922_016__phrase_p123
8,016,_sent_p123,1,3,2.0,5.0,1.0,6.0,[sentence],6.0,2025-01-01,"[category1, category2, pinyin]","[True, True, True]",,None,,1,1,None,None,1000000,None,0922_016__sent_p123
9,016,_propern_p123,1,3,2.0,5.0,1.0,6.0,[proper noun],6.0,2025-01-01,"[category1, category2, pinyin]","[True, True, True]",,None,,1,1,None,None,1000000,None,0922_016__propern_p123


# 1. Load data

In [16]:
truly_load_data = True
if truly_load_data:
    df_all_vocab = load_raw_data()
    df_all_vocab.to_csv('latest_data.csv', index=False)
else:
    df_all_vocab = pd.read_csv('latest_data.csv')
    print('!!!!!!!! WARNING: not truly loading data !!!!!!!!')

df_dups = check_dups(df_all_vocab)
print(f'# duplicate vocab: {len(df_dups)}')
print(df_all_vocab.shape)
df_all_vocab.head(3)

# duplicate vocab: 0
(5330, 28)


,id,chinese,pinyin,english,type,priority,known,known_pinyin_prompt,known_english_prompt,phonetic,category1,category2,quality,word1,word1_english,word2,word2_english,word3,word3_english,word4,word4_english,sentence,sentence_pinyin,sentence_english,date,cat1,per,adu
0,1,房贷,fáng dài,mortgage,combo,1.0,1.0,1.0,2.0,NaN,life,NaN,1.0,房,house,贷,loan,NaN,NaN,NaN,NaN,他每个月都要还房贷,Ta měi gè yuè dōu yào huán fángdài,He has to pay his mortgage every month,2025-01-02,NaN,5.0,5.0
1,2,白天,bái tiān,daytime,combo,2.0,2.0,1.0,1.0,NaN,time,NaN,1.0,白,white,天,day,NaN,NaN,NaN,NaN,白天很热晚上比较凉快,Báitiān hěn rè wǎnshàng bǐjiào liángkuai,It is hot in the daytime and cooler at night,2025-01-02,NaN,5.0,5.0
2,4,组合,zǔ hé,combination,combo,1.0,5.0,5.0,4.0,NaN,general,NaN,2.0,组,set,合适,suitable,NaN,NaN,NaN,NaN,这个组合在比赛中获得了第一名。,Zhège zǔhé zài bǐsài zhōng huòdé le dì yī míng.,This group won first place in the competition.,2025-01-02,NaN,5.0,5.0


# 2. Select data

In [17]:
dfs_vocab_lists = {}
n_vocab_by_recording = {}
for i_rrow, rrow in df_all_recordings_tomake.iterrows():
    dfs_vocab_lists[rrow['recording_name']] = filter_df_to_vocab_of_interest(df_all_vocab, rrow)
    n_vocab_by_recording[rrow['recording_name']] = len(dfs_vocab_lists[rrow['recording_name']])
    print(rrow['recording_name'], n_vocab_by_recording[rrow['recording_name']])

dfs_vocab_lists[rrow['recording_name']].head(3)

0922_012_efirst_csent_p1_1 75
0922_012_efirst_csent_p2_2 616
0922_012_efirst_csent_p3_3 388
0922_012_efirst_csent_p1_3_known2 435
0922_001_full_word_sent_p1_1 43
0922_001_full_word_sent_p2_2 364
0922_001_full_word_sent_p3_3 903
0922_004_word_only_p1_1 43
0922_004_word_only_p2_2 364
0922_004_word_only_p1_2_phrase 131
0922_004_word_only_p3_3_phrase 70
0922_016_efirst_word_only_p1_3_phrase_p1-3 165
0922_016_efirst_word_only_p1_3_sent_p1-3 58
0922_016_efirst_word_only_p1_3_propern_p1-3 82
0922_013_ccombo_sent_p1_1_shortcombos 9
0922_013_ccombo_sent_p1_4_food_yt 93
0922_013_ccombo_sent_p1_4_animal_yt 18
0922_006_ecombo_p1_2 868
0922_015_conly_sent_p1_2_known2 654
0922_001_full_word_sent_p1_3_recentAug15 212


,id,chinese,pinyin,english,type,priority,known,known_pinyin_prompt,known_english_prompt,phonetic,category1,category2,quality,word1,word1_english,word2,word2_english,word3,word3_english,word4,word4_english,sentence,sentence_pinyin,sentence_english,date,cat1,per,adu
0,4595,宽,kuān,wide,single char,1.0,5.0,2.0,2.0,NaN,amount,degree,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,这条路很宽,zhè tiáo lù hěn kuān,this road is wide,2025-08-21,daily add,5.0,5.0
1,4596,窄,zhǎi,narrow,single char,1.0,5.0,3.0,2.0,NaN,amount,degree,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,这个走廊太窄了,zhè gè zǒu láng tài zhǎi le,this corridor is too narrow,2025-08-21,daily add,5.0,5.0
2,5307,后端,hòu duān,back end (engineer),combo,2.0,5.0,5.0,5.0,NaN,career,NaN,3.0,后,back,极端,extreme,NaN,NaN,NaN,NaN,动物医院的后端系统记录了每只猫狗的病历,dòngwù yīyuàn de hòuduān xìtǒng jìlù...,The animal hospital backend system recorded ea...,2025-09-16,daily add,2.0,5.0


# 3. Create vocabulary audio recordings

In [18]:
for i_rrow, rrow in df_all_recordings_tomake.iterrows():
    if rrow['recording_id'] == '005':
        dfs_vocab_lists[rrow['recording_name']] = compute_pinyin_and_create_recordings(dfs_vocab_lists[rrow['recording_name']])
    for i_row, row in dfs_vocab_lists[rrow['recording_name']].iterrows():
        print(f'{rrow['recording_name']} row{i_row}')
        create_tts_files_for_one_vocab_word(row, rrow)

0922_012_efirst_csent_p1_1 row0
0.000s, ALREADY EXISTS, zh, 详细
0.000s, ALREADY EXISTS, zh_slow, 详细
0.000s, ALREADY EXISTS, english, detailed
0.000s, ALREADY EXISTS, zh, 请给我详细地址
0922_012_efirst_csent_p1_1 row1
0.000s, ALREADY EXISTS, zh, 通常
0.000s, ALREADY EXISTS, zh_slow, 通常
0.000s, ALREADY EXISTS, english, generally;usually
0.000s, ALREADY EXISTS, zh, 我通常早起
0922_012_efirst_csent_p1_1 row2
0.000s, ALREADY EXISTS, zh, 退出
0.000s, ALREADY EXISTS, zh_slow, 退出
0.000s, ALREADY EXISTS, english, quit
0.000s, ALREADY EXISTS, zh, 我决定退出比赛
0922_012_efirst_csent_p1_1 row3
0.000s, ALREADY EXISTS, zh, 投资
0.000s, ALREADY EXISTS, zh_slow, 投资
0.000s, ALREADY EXISTS, english, to invest
0.000s, ALREADY EXISTS, zh, 他投资了一家公司
0922_012_efirst_csent_p1_1 row4
0.000s, ALREADY EXISTS, zh, 降低
0.000s, ALREADY EXISTS, zh_slow, 降低
0.000s, ALREADY EXISTS, english, to lower;reduce
0.000s, ALREADY EXISTS, zh, 我们需要降低成本
0922_012_efirst_csent_p1_1 row5
0.000s, ALREADY EXISTS, zh, 上升
0.000s, ALREADY EXISTS, zh_slow, 上升
0.0

# 4. Combine audio to one file for each vocab word

In [ ]:
df_vocab_audio_durations = {}
for i_rrow, rrow in df_all_recordings_tomake.iterrows():
    df_vocab_audio_durations[rrow['recording_name']] = combine_audio_files_and_compute_durations(dfs_vocab_lists[rrow['recording_name']], rrow, making_video=False)
    df_vocab_audio_durations[rrow['recording_name']].to_csv(f"{audio_final_path}{rrow['recording_name']}.csv", index=False)
df_vocab_audio_durations[rrow['recording_name']].head(3)

0.53 seconds, recid0922_012_efirst_csent_p1_1, row 0, 详细
0.36 seconds, recid0922_012_efirst_csent_p1_1, row 1, 通常
0.35 seconds, recid0922_012_efirst_csent_p1_1, row 2, 退出
0.35 seconds, recid0922_012_efirst_csent_p1_1, row 3, 投资
0.36 seconds, recid0922_012_efirst_csent_p1_1, row 4, 降低
0.36 seconds, recid0922_012_efirst_csent_p1_1, row 5, 上升
0.37 seconds, recid0922_012_efirst_csent_p1_1, row 6, 提高
0.37 seconds, recid0922_012_efirst_csent_p1_1, row 7, 好客
0.37 seconds, recid0922_012_efirst_csent_p1_1, row 8, 方言
0.37 seconds, recid0922_012_efirst_csent_p1_1, row 9, 王朝
0.37 seconds, recid0922_012_efirst_csent_p1_1, row 10, 充电器
0.36 seconds, recid0922_012_efirst_csent_p1_1, row 11, 登出
0.37 seconds, recid0922_012_efirst_csent_p1_1, row 12, 登录
0.36 seconds, recid0922_012_efirst_csent_p1_1, row 13, 风扇
0.37 seconds, recid0922_012_efirst_csent_p1_1, row 14, 计算机
0.36 seconds, recid0922_012_efirst_csent_p1_1, row 15, 流量
0.36 seconds, recid0922_012_efirst_csent_p1_1, row 16, 能源
0.36 seconds, recid092

,chinese,pinyin,english,sentence,sentence_pinyin,sentence_english,d_chinese,d_chinese_slow,d_english,d_sent,d_sent_english,rel_start_chinese,rel_start_english,rel_start_sent,rel_start_sent_english,sum_theory,combined
0,宽,kuān,wide,这条路很宽,zhè tiáo lù hěn kuān,this road is wide,0.936,1.056,0.840,1.848,1.536,0,2.992,4.332,6.680,11.564,11.563292
1,窄,zhǎi,narrow,这个走廊太窄了,zhè gè zǒu láng tài zhǎi le,this corridor is too narrow,0.960,1.104,0.888,2.352,2.256,0,3.064,4.452,7.304,13.412,13.411292
2,后端,hòu duān,back end (engineer),动物医院的后端系统记录了每只猫狗的病历,dòngwù yīyuàn de hòuduān xìtǒng jìlù...,The animal hospital backend system recorded ea...,1.152,1.320,1.944,5.448,5.712,0,3.472,5.916,11.864,24.524,24.523292


# 5. Combine individual vocab into final audio recording

In [20]:
for i_rrow, rrow in df_all_recordings_tomake.iterrows():
    start_time = time.time()
    # Construct list of individual audio files
    all_audio_files = []
    for i_row, row in df_vocab_audio_durations[rrow['recording_name']].iterrows():
       all_audio_files.append(AudioSegment.from_mp3(f"audio_files/rows/{rrow['recording_id']}/{row['chinese']}.mp3"))

    # Construct and export whole audio file
    audio_concat = all_audio_files[0]
    for audio in all_audio_files[1:]:
        audio_concat += audio
    audio_concat.export(f"{audio_final_path}{rrow['recording_name']}.mp3", format="mp3")
    print(f"{(time.time()-start_time):.2f}s, {audio_final_path}{rrow['recording_name']}.mp3")

6.93s, audio_files/products/audio_refresh/20250922/0922_012_efirst_csent_p1_1.mp3
66.80s, audio_files/products/audio_refresh/20250922/0922_012_efirst_csent_p2_2.mp3
41.44s, audio_files/products/audio_refresh/20250922/0922_012_efirst_csent_p3_3.mp3
45.91s, audio_files/products/audio_refresh/20250922/0922_012_efirst_csent_p1_3_known2.mp3
4.97s, audio_files/products/audio_refresh/20250922/0922_001_full_word_sent_p1_1.mp3
48.19s, audio_files/products/audio_refresh/20250922/0922_001_full_word_sent_p2_2.mp3
150.96s, audio_files/products/audio_refresh/20250922/0922_001_full_word_sent_p3_3.mp3
4.25s, audio_files/products/audio_refresh/20250922/0922_004_word_only_p1_1.mp3
37.45s, audio_files/products/audio_refresh/20250922/0922_004_word_only_p2_2.mp3
12.62s, audio_files/products/audio_refresh/20250922/0922_004_word_only_p1_2_phrase.mp3
6.84s, audio_files/products/audio_refresh/20250922/0922_004_word_only_p3_3_phrase.mp3
16.76s, audio_files/products/audio_refresh/20250922/0922_016_efirst_word_on